In [3]:
# ============================================================================
# TDA TRADING STRATEGY - PHASE 3: PERSISTENT HOMOLOGY
# ============================================================================

import subprocess
import sys
import os

print("=" * 70)
print("TDA TRADING STRATEGY - PHASE 3: PERSISTENT HOMOLOGY")
print("=" * 70)

# ============================================================================
# FIX NUMPY COMPATIBILITY (First run only)
# ============================================================================

try:
    from ripser import ripser
    print("\n✅ Ripser already installed!")
except (ImportError, ValueError):
    print("\n🔧 Installing dependencies with correct versions...")
    subprocess.run([sys.executable, '-m', 'pip', 'uninstall', 'numpy', '-y', '-q'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'numpy==1.24.3', '-q'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'scikit-learn==1.3.0', '-q'])
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'ripser', '-q'])
    print("✅ Installation complete!")
    print("\n⚠️  Please click 'Runtime' → 'Restart runtime' in menu above")
    print("Then run this cell again.")
    raise SystemExit("Restart required - run this cell again after restart")

# ============================================================================
# IMPORTS
# ============================================================================

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

# ============================================================================
# DATA SETUP
# ============================================================================

if not os.path.exists('stock_returns.csv'):
    print("\n⚠️  Downloading data...")
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'yfinance', '-q'])
    import yfinance as yf

    universe = [
        'AAPL', 'MSFT', 'AMZN', 'NVDA', 'META', 'GOOG', 'TSLA',
        'NFLX', 'JPM', 'PEP', 'CSCO', 'ORCL', 'DIS', 'BAC',
        'XOM', 'IBM', 'INTC', 'AMD', 'KO', 'WMT'
    ]

    start_date = '2019-01-01'
    end_date = '2024-12-10'

    prices_dict = {}
    for ticker in universe:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date, end=end_date)
            if not hist.empty and 'Close' in hist.columns:
                prices_dict[ticker] = hist['Close']
        except:
            pass

    prices = pd.DataFrame(prices_dict)
    prices = prices.fillna(method='ffill').fillna(method='bfill')
    returns = prices.pct_change().dropna()

    prices.to_csv('stock_prices.csv')
    returns.to_csv('stock_returns.csv')
    print("✅ Data downloaded!")
else:
    print("\n✅ Data files found!")

print("\n📂 Loading data...")
returns = pd.read_csv('stock_returns.csv', index_col=0, parse_dates=True)
print(f"✅ Loaded {len(returns)} days × {len(returns.columns)} stocks")

# ============================================================================
# STEP 1: CORRELATION DISTANCE
# ============================================================================

print("\n" + "=" * 70)
print("STEP 1: CORRELATION DISTANCE METRIC")
print("=" * 70)

print("\n📐 Distance formula: d_ij = sqrt(2(1 - ρ_ij))")

LOOKBACK = 60
recent_returns = returns.tail(LOOKBACK)
corr_matrix = recent_returns.corr()

distance_matrix = np.sqrt(2 * (1 - corr_matrix.values))
np.fill_diagonal(distance_matrix, 0)

print(f"\n✅ Distance matrix created")
print(f"Min distance: {distance_matrix[distance_matrix > 0].min():.3f}")
print(f"Max distance: {distance_matrix.max():.3f}")
print(f"Mean distance: {distance_matrix[distance_matrix > 0].mean():.3f}")

# ============================================================================
# STEP 2: SINGLE PERSISTENCE DIAGRAM
# ============================================================================

print("\n" + "=" * 70)
print("STEP 2: PERSISTENCE DIAGRAM")
print("=" * 70)

print(f"\n⏳ Computing persistent homology on last {LOOKBACK} days...")

result = ripser(distance_matrix, maxdim=1, distance_matrix=True)
diagrams = result['dgms']

print("✅ Persistence computed!")
print(f"H0 (components): {len(diagrams[0])} features")
print(f"H1 (loops): {len(diagrams[1])} features")

# Calculate Betti numbers at different epsilon values
def calculate_betti_numbers(diagram, epsilon_values):
    """Calculate Betti numbers at specific epsilon values"""
    betti = []
    for eps in epsilon_values:
        # Count features that exist at this epsilon (birth <= eps < death)
        count = np.sum((diagram[:, 0] <= eps) & (diagram[:, 1] > eps))
        betti.append(count)
    return np.array(betti)

epsilon_values = np.linspace(0, distance_matrix.max(), 100)
betti_0 = calculate_betti_numbers(diagrams[0], epsilon_values)
betti_1 = calculate_betti_numbers(diagrams[1], epsilon_values)

# Plot Betti curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))

ax1.plot(epsilon_values, betti_0, linewidth=2, color='steelblue')
ax1.fill_between(epsilon_values, betti_0, alpha=0.3, color='steelblue')
ax1.set_xlabel('ε (distance threshold)', fontsize=12)
ax1.set_ylabel('β₀ (# components)', fontsize=12)
ax1.set_title(f'Betti-0 Curve (Last {LOOKBACK} Days)', fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)

ax2.plot(epsilon_values, betti_1, linewidth=2, color='darkorange')
ax2.fill_between(epsilon_values, betti_1, alpha=0.3, color='darkorange')
ax2.set_xlabel('ε (distance threshold)', fontsize=12)
ax2.set_ylabel('β₁ (# loops)', fontsize=12)
ax2.set_title(f'Betti-1 Curve (Last {LOOKBACK} Days)', fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('betti_curves_example.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n💾 Saved: betti_curves_example.png")

print(f"\n📊 Interpretation:")
print(f"  β₀: starts at {int(betti_0[0])} (all isolated) → ends at {int(betti_0[-1])} (connected)")
print(f"  β₁: max = {int(betti_1.max())} loops detected")
print(f"  β₁: persists over {(betti_1 > 0).sum()}/{len(betti_1)} points")

# ============================================================================
# STEP 3: HISTORICAL TOPOLOGY FEATURES
# ============================================================================

print("\n" + "=" * 70)
print("STEP 3: HISTORICAL TOPOLOGY FEATURES")
print("=" * 70)

print(f"\n⏳ Calculating topology features for {len(returns) - LOOKBACK} days...")
print("This takes 2-3 minutes...")

def calculate_topology_features(returns_df, window=60):
    """Calculate topological features over rolling windows"""

    dates = []
    h1_loop_count = []
    h1_total_lifetime = []

    for i in range(window, len(returns_df)):
        if i % 200 == 0:
            print(f"  Progress: {i}/{len(returns_df)}")

        # Calculate correlation
        returns_window = returns_df.iloc[i-window:i]
        corr = returns_window.corr()

        # Distance matrix
        dist = np.sqrt(2 * (1 - corr.values))
        np.fill_diagonal(dist, 0)

        # Compute persistence
        try:
            result = ripser(dist, maxdim=1, distance_matrix=True)
            h1_diagram = result['dgms'][1]

            # Remove infinite bars (shouldn't have any, but just in case)
            h1_diagram = h1_diagram[~np.isinf(h1_diagram).any(axis=1)]

            dates.append(returns_df.index[i])
            h1_loop_count.append(len(h1_diagram))

            # Calculate total persistence (sum of lifetimes)
            if len(h1_diagram) > 0:
                lifetimes = h1_diagram[:, 1] - h1_diagram[:, 0]
                h1_total_lifetime.append(lifetimes.sum())
            else:
                h1_total_lifetime.append(0)

        except Exception as e:
            # If calculation fails, use previous values
            if len(dates) > 0:
                dates.append(returns_df.index[i])
                h1_loop_count.append(h1_loop_count[-1])
                h1_total_lifetime.append(h1_total_lifetime[-1])

    return pd.DataFrame({
        'h1_loops': h1_loop_count,
        'h1_persistence': h1_total_lifetime
    }, index=dates)

# Calculate topology features
topology_ts = calculate_topology_features(returns, window=LOOKBACK)

print(f"\n✅ Calculated topology features for {len(topology_ts)} days")
print(f"Date range: {topology_ts.index[0].date()} to {topology_ts.index[-1].date()}")

# Save
topology_ts.to_csv('topology_features.csv')
print("💾 Saved: topology_features.csv")

# ============================================================================
# STEP 4: VISUALIZE TOPOLOGY EVOLUTION
# ============================================================================

print("\n" + "=" * 70)
print("STEP 4: TOPOLOGY EVOLUTION OVER TIME")
print("=" * 70)

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Crisis periods for reference
crisis_periods = [
    ('2020-02-01', '2020-04-01', 'COVID Crash', 'red'),
    ('2022-01-01', '2022-06-01', 'Fed Pivot', 'orange'),
    ('2023-10-01', '2024-12-01', 'AI Bubble', 'purple')
]

# Plot H1 loop count
ax = axes[0]
topology_ts['h1_loops'].plot(ax=ax, linewidth=1.5, color='darkorange', label='# of loops (H1)')
for start, end, label, color in crisis_periods:
    ax.axvspan(pd.to_datetime(start), pd.to_datetime(end),
               alpha=0.15, color=color, label=label if ax == axes[0] else '')
ax.set_ylabel('# of Loops', fontsize=12, fontweight='bold')
ax.set_title('Market Complexity Over Time (Higher = More Circular Dependencies)', fontsize=14, fontweight='bold')
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

# Plot H1 persistence
ax = axes[1]
topology_ts['h1_persistence'].plot(ax=ax, linewidth=1.5, color='darkgreen', label='H1 persistence')
for start, end, label, color in crisis_periods:
    ax.axvspan(pd.to_datetime(start), pd.to_datetime(end), alpha=0.15, color=color)
ax.set_ylabel('Total Loop Lifetime', fontsize=12, fontweight='bold')
ax.set_title('Loop Stability Over Time (Higher = Stable Circular Relationships)', fontsize=14, fontweight='bold')
ax.set_xlabel('Date', fontsize=12)
ax.legend(loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('topology_evolution.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n💾 Saved: topology_evolution.png")

# ============================================================================
# STEP 5: TOPOLOGY-BASED REGIME CLASSIFIER
# ============================================================================

print("\n" + "=" * 70)
print("STEP 5: TOPOLOGY-BASED REGIME CLASSIFICATION")
print("=" * 70)

# Calculate topology volatility (instability measure)
topology_ts['topology_volatility'] = (
    topology_ts['h1_loops'].rolling(30).std() +
    topology_ts['h1_persistence'].rolling(30).std()
)

# Classify regimes based on 75th percentile threshold
threshold = topology_ts['topology_volatility'].quantile(0.75)
topology_ts['regime'] = 'stable'
topology_ts.loc[topology_ts['topology_volatility'] > threshold, 'regime'] = 'unstable'

print(f"\n📊 Regime Classification:")
print(f"Instability threshold: {threshold:.3f}")
print(f"Stable days: {(topology_ts['regime'] == 'stable').sum()} ({(topology_ts['regime'] == 'stable').mean():.1%})")
print(f"Unstable days: {(topology_ts['regime'] == 'unstable').sum()} ({(topology_ts['regime'] == 'unstable').mean():.1%})")

# Visualize regime classification
plt.figure(figsize=(16, 6))
topology_ts['topology_volatility'].plot(linewidth=1.5, color='darkblue', label='Topology Volatility')
plt.axhline(y=threshold, color='red', linestyle='--', linewidth=2, label='Instability Threshold')

# Shade unstable periods in red
unstable_periods = topology_ts[topology_ts['regime'] == 'unstable']
for date in unstable_periods.index:
    plt.axvspan(date, date + pd.Timedelta(days=1), alpha=0.3, color='red')

# Add crisis period overlays
for start, end, label, color in crisis_periods:
    plt.axvspan(pd.to_datetime(start), pd.to_datetime(end),
                alpha=0.1, color=color, label=label)

plt.ylabel('Topology Volatility', fontsize=12, fontweight='bold')
plt.xlabel('Date', fontsize=12)
plt.title('Topology-Based Regime Detection (Red Shading = Don\'t Trade)', fontsize=14, fontweight='bold')
plt.legend(loc='upper left')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('topology_regimes.png', dpi=150, bbox_inches='tight')
plt.show()
print("\n💾 Saved: topology_regimes.png")

# Save regime classification
topology_ts.to_csv('topology_regimes.csv')
print("💾 Saved: topology_regimes.csv")

# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

print("\n" + "=" * 70)
print("SUMMARY STATISTICS")
print("=" * 70)

print(f"\nTopology Features:")
print(f"  Average # loops: {topology_ts['h1_loops'].mean():.2f}")
print(f"  Max # loops: {topology_ts['h1_loops'].max():.0f}")
print(f"  Average persistence: {topology_ts['h1_persistence'].mean():.3f}")

print(f"\nRegime Analysis:")
unstable_returns = returns.loc[topology_ts[topology_ts['regime'] == 'unstable'].index]
stable_returns = returns.loc[topology_ts[topology_ts['regime'] == 'stable'].index]

if len(unstable_returns) > 0 and len(stable_returns) > 0:
    print(f"  Volatility during unstable periods: {unstable_returns.std().mean():.4f}")
    print(f"  Volatility during stable periods: {stable_returns.std().mean():.4f}")
    print(f"  Ratio: {unstable_returns.std().mean() / stable_returns.std().mean():.2f}x")

print("\n" + "=" * 70)
print("✅ PHASE 3 COMPLETE!")
print("=" * 70)
print("\nWhat we built:")
print("  ✅ Persistent homology computation")
print("  ✅ Betti curves (β₀ and β₁)")
print("  ✅ Historical topology evolution (2019-2024)")
print("  ✅ Topology-based regime classifier")
print("\nFiles created:")
print("  📊 betti_curves_example.png")
print("  📊 topology_evolution.png")
print("  📊 topology_regimes.png")
print("  💾 topology_features.csv")
print("  💾 topology_regimes.csv")
print("\n🎯 Next: Phase 4 - Strategy Backtest (combine residuals + topology)")
print("=" * 70)

TDA TRADING STRATEGY - PHASE 3: PERSISTENT HOMOLOGY

🔧 Installing dependencies with correct versions...


ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



✅ Installation complete!

⚠️  Please click 'Runtime' → 'Restart runtime' in menu above
Then run this cell again.
Traceback (most recent call last):
  File "/tmp/ipython-input-925037856.py", line 18, in <cell line: 0>
    from ripser import ripser
  File "/usr/local/lib/python3.12/dist-packages/ripser/__init__.py", line 1, in <module>
    from .ripser import Rips, ripser, lower_star_img
  File "/usr/local/lib/python3.12/dist-packages/ripser/ripser.py", line 27, in <module>
    from scipy import sparse
  File "/usr/local/lib/python3.12/dist-packages/scipy/__init__.py", line 131, in __getattr__
    return _importlib.import_module(f'scipy.{name}')
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/importlib/__init__.py", line 90, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.12/dist-packages/scipy/sparse/__init__.py", line 304, in <

TypeError: object of type 'NoneType' has no len()